# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM
## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during
 the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [56]:
#importing all necessary libraries
import requests
import json
import pandas as pd
from pprint import pprint
from bs4 import BeautifulSoup
#requesting page with first 200 universities
r = requests.get('https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json')
#creating parser
data=json.loads(r.text)['data'][:200]
#declaring the dataframe
times=pd.DataFrame({'Name' : [], 
                    'Rank':[], 
                    'Country':[], 
                    'Region':[], 
                    'FM Total':[],
                    'FM International':[],
                    'Students Total':[],
                    'Students International':[]}) #TODO region and international faculty member is left

def word_checker(word):
    if word=='Shi': return False
    if any(c.isdigit() for c in word):
        return False
    elif any(c.islower() for c in word):
        return True
    else: return False

def region_finder(address):
    addr_segments=address.split(',')
    region=addr_segments[-1].replace(' ','')
    addr_segments=addr_segments[0:-1]
    for addr_segment in addr_segments:
        if any(c.islower() for c in addr_segment):
            tmp=""
            for word in addr_segment.split(' '):
                if word_checker(word): tmp=tmp+' '+word
            region=tmp
    return region.lstrip().replace('\n','')

idx=1
for row in data:
    rank=int(row['rank'].replace("=",""))
    name=row['name']
    country=row['location']
    students=int(row['stats_number_students'].replace(",",""))
    ratio=float(row['stats_student_staff_ratio'])
    percent_i=float(row['stats_pc_intl_students'].replace("%",""))
    
    region=country
    r=requests.get('https://www.timeshighereducation.com'+row['url'])
    soup=BeautifulSoup(r.text, "html5lib") 
    address=soup.find("div",class_="profile-card__address")
    if address==None:
        website=soup.find("div",class_="profile-card__website").text
        r=requests.get(website)
        soup=BeautifulSoup(r.text, "html5lib")
        region=soup.find("span",class_="region").text
    else:
        region=region_finder(address.text)
    times.loc[idx]=[country,None,int(students/ratio),name,rank,region,students*percent_i/100,students]
    idx+=1   
times

                Country FM International  FM Total  \
1        United Kingdom             None    1822.0   
2        United Kingdom             None    1687.0   
3         United States             None     339.0   
4         United States             None    2112.0   
5         United States             None    1284.0   
6         United States             None    2283.0   
7         United States             None     958.0   
8        United Kingdom             None    1390.0   
9         United States             None    2181.0   
10          Switzerland             None    1317.0   
11        United States             None    3132.0   
12        United States             None    2826.0   
13        United States             None    3604.0   
14        United States             None    4358.0   
15        United States             None    4091.0   
16       United Kingdom             None    2886.0   
17        United States             None    3390.0   
18        United States     

In [60]:
# (a) ratio between faculty members and students, (b) ratio of international students?
# Answer the previous question aggregating the data by (c) country and (d) region.
times=backup
def sorter(data,column1,ratio=None,column2=None,filterby=None,filter_val=None,ascending=False):
    result=data.copy(True)
    if filterby!=None:
        result=data[data[filterby]==filter_val].copy(True)
    if ratio!=None:
        result[ratio]=result[column1].divide(result[column2])
        result=result.sort_values(ratio,ascending=ascending)
    else:
        result=result.sort_values(column1,ascending=ascending)
    return result

sorter(times,'FM Total','fm/student','Students Total')

,Country,FM International,FM Total,Name,Rank,Region,Students International,Students Total,fm/student
106,United States,None,3639.0,Vanderbilt University,105.0,Tennessee,1561.43,12011.0,0.302972
110,Denmark,None,7413.0,University of Copenhagen,109.0,København,4255.30,30395.0,0.243889
13,United States,None,3604.0,Johns Hopkins University,13.0,Maryland,3719.52,15498.0,0.232546
12,United States,None,2826.0,Yale University,12.0,Connecticut,2552.55,12155.0,0.232497
154,United States,None,2240.0,University of Rochester,153.0,New York,2794.44,9636.0,0.232462
98,United States,None,2874.0,Emory University,98.0,Georgia,2403.31,12649.0,0.227212
17,United States,None,3390.0,Duke University,17.0,North Carolina,3356.32,15256.0,0.222208
115,France,None,548.0,École Polytechnique,115.0,Palaiseau,1007.64,2799.0,0.195784
184,Italy,None,99.0,Scuola Normale Superiore di Pisa,184.0,Pisa,36.05,515.0,0.192233
186,United States,None,2688.0,University of Miami,186.0,Florida,2537.60,15860.0,0.169483


In [58]:
sorter(times,'Students International','intl/student','Students Total')

,Country,FM International,FM Total,Name,Rank,Region,Students International,Students Total,intl/student
25,United Kingdom,None,825.0,London School of Economics and Political Science,25.0,London,7146.15,10065.0,0.71
179,Luxembourg,None,340.0,University of Luxembourg,179.0,Esch-sur-Alzette,2832.33,4969.0,0.57
8,United Kingdom,None,1390.0,Imperial College London,8.0,London,8721.35,15857.0,0.55
38,Switzerland,None,886.0,École Polytechnique Fédérale de Lausanne,38.0,Lausanne,5460.40,9928.0,0.55
103,Netherlands,None,929.0,Maastricht University,103.0,Maastricht,8363.50,16727.0,0.50
16,United Kingdom,None,2886.0,University College London,16.0,London,14848.96,30304.0,0.49
144,United Kingdom,None,692.0,University of St Andrews,143.0,Fife,4322.40,9005.0,0.48
24,United States,None,938.0,Carnegie Mellon University,24.0,Pennsylvania,5704.20,12676.0,0.45
121,United Kingdom,None,1158.0,Queen Mary University of London,121.0,London,6932.25,15405.0,0.45
40,Hong Kong,None,1020.0,University of Hong Kong,40.0,Hong Kong Island,7712.88,18364.0,0.42


In [61]:
sorter(times,'FM Total','fm/student','Students Total','Country','United States')

,Country,FM International,FM Total,Name,Rank,Region,Students International,Students Total,fm/student
106,United States,None,3639.0,Vanderbilt University,105.0,Tennessee,1561.43,12011.0,0.302972
13,United States,None,3604.0,Johns Hopkins University,13.0,Maryland,3719.52,15498.0,0.232546
12,United States,None,2826.0,Yale University,12.0,Connecticut,2552.55,12155.0,0.232497
154,United States,None,2240.0,University of Rochester,153.0,New York,2794.44,9636.0,0.232462
98,United States,None,2874.0,Emory University,98.0,Georgia,2403.31,12649.0,0.227212
17,United States,None,3390.0,Duke University,17.0,North Carolina,3356.32,15256.0,0.222208
186,United States,None,2688.0,University of Miami,186.0,Florida,2537.60,15860.0,0.169483
102,United States,None,4468.0,University of Pittsburgh,100.0,Pennsylvania,2636.70,26367.0,0.169454
14,United States,None,4358.0,Columbia University,14.0,New York,8507.84,26587.0,0.163915
9,United States,None,2181.0,University of Chicago,9.0,Illinois,3381.25,13525.0,0.161257
